In [ ]:
import requests
from lxml import html
import re
import csv
import pandas as pd

In [ ]:
def read_url(url):
    url = "http://" + url
    page = requests.get(url)
    content = str(page.content)
    w_page = html.fromstring(content)
    w_pages = w_page.xpath("//a/@href")
    
    title_start = re.search('<title>', content).end()
    content_title_1 = content[title_start:]
    title_end = re.search('</title>', content_title_1).start()
    content_title_2 = content_title_1[:title_end]
    company_name = content_title_2
    
    return company_name

In [ ]:
print(read_url("www.baeckerei-grobe.de/"))

In [ ]:
data = pd.read_csv('20181018 Domain+WZ2008 Hackathon INOBAS.csv', sep=';', encoding='latin1', names=['Domain', 'Section', 'Code'])
# print(data.head())

In [ ]:
company_urls = []

for index, row in data.iterrows():
#     print(index, row['Domain'])
    company_urls.append(row['Domain'])

In [ ]:
print(len(company_urls))

In [ ]:
for url in company_urls[1:20]:
    print(read_url(url))

In [ ]:
def check_string_in_list(string, arr):
    arr = list(set(arr))
    
    for item in arr:
        if string in item:
            return item

In [ ]:
def get_about(url):
    url = "http://" + url
    page = requests.get(url)
    content = str(page.content)
    w_page = html.fromstring(content)
    w_pages = w_page.xpath("//a/@href")
    
    # Problem iterating 3 times TODO
    facebook = check_string_in_list("facebook", w_pages)
    instagram = check_string_in_list("instagram", w_pages)
    twitter = check_string_in_list("twitter", w_pages)
    impressium = check_string_in_list("impressum", w_pages)
    
    return [facebook, instagram, twitter, url+impressium]

In [ ]:
baeckerei_grobe = get_about("www.baeckerei-grobe.de/")
print(baeckerei_grobe)

# TODO remove ../ from urls

In [ ]:
def get_content(url):
    page = requests.get(url)
    content = str(page.content)
    w_page = html.fromstring(content)
    w_pages = w_page.xpath("//a/@href")
    
    return content

In [ ]:
print(get_content("http://www.baeckerei-grobe.de/pages/impressum.php"))